In [2]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

bucket = "sagemaker-{}-{}".format(region, account_id)
prefix = "demo-multimodel-endpoint"

role = "arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032"



In [4]:
from time import gmtime, strftime

model_name = "Monai-DEMO-Inference" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz"
container = "617011600974.dkr.ecr.eu-west-1.amazonaws.com/monai-braintumor-segmentation:latest"

print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + container)

container = {"Image": container, "ModelDataUrl": model_url, "Mode": "MultiModel"}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: Monai-DEMO-Inference2022-06-08-12-27-35
Model data Url: s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz
Container image: 617011600974.dkr.ecr.eu-west-1.amazonaws.com/monai-braintumor-segmentation:latest
Model Arn: arn:aws:sagemaker:eu-west-1:617011600974:model/monai-demo-inference2022-06-08-12-27-35


In [5]:
endpoint_config_name = "Monai-DEMO-MultiModelEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint config name: Monai-DEMO-MultiModelEndpointConfig-2022-06-08-12-28-51
Endpoint config Arn: arn:aws:sagemaker:eu-west-1:617011600974:endpoint-config/monai-demo-multimodelendpointconfig-2022-06-08-12-28-51


In [7]:
import time

endpoint_name = "Monai-DEMO-MultiModelEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

Endpoint name: Monai-DEMO-MultiModelEndpoint-2022-06-08-12-29-21
Endpoint Arn: arn:aws:sagemaker:eu-west-1:617011600974:endpoint/monai-demo-multimodelendpoint-2022-06-08-12-29-21
Endpoint Status: Creating
Waiting for Monai-DEMO-MultiModelEndpoint-2022-06-08-12-29-21 endpoint to be in service...
